* https://github.com/JuliaMath/Combinatorics.jl/blob/master/src/permutations.jl#L47
* https://discourse.julialang.org/t/is-there-a-function-behaving-the-same-as-next-permutation-does-in-c/63451/13

In [1]:
module O

function next_perm!(v::AbstractVector)
    length(v) ≤ 1 && return false
    
    k = findlast(isless(v[i], v[i+1]) for i in firstindex(v):lastindex(v)-1)
    isnothing(k) && (reverse!(v); return false)
    i = k + firstindex(v)-1
    
    j = findlast(isless(v[i], v[j]) for j in i+1:lastindex(v)) + i
    v[i], v[j] = v[j], v[i]
    reverse!(v, i + 1)
    return true
end

function collect_perm!(v::AbstractVector)
    a = [copy(v)]
    while next_perm!(v) push!(a, copy(v)) end
    a
end

struct Perm{T<:AbstractVector} v::T end
perm(v::AbstractVector; kwargs...) = Perm(sort(v; kwargs...))

function Base.length(p::Perm)
    v = p.v
    N = factorial(length(v))
    i = firstindex(v)
    for j in eachindex(v)
        if isless(v[i], v[j])
            N = N ÷ factorial(j-i)
            i = j
        end
    end
    N = N ÷ factorial(lastindex(v)-i+1)
    N
end

Base.eltype(p::Perm{T}) where T<:AbstractVector = T

function Base.iterate(p::Perm)
    s = copy(p.v)
    t = copy(s)
    next_perm!(t) || return (s, nothing)
    u = copy(t)
    next_perm!(u) || return (s, (t, nothing))
    return (s, (t, u))
end

function Base.iterate(p::Perm, state)
    isnothing(state) && return nothing
    s, t = state
    isnothing(t) && return (s, nothing)
    u = copy(t)
    next_perm!(u) || return (s, (t, nothing))
    return (s, (t, u))
end

#function Base.iterate(p::Perm, s = copy(p.v))
#    isnothing(s) && return nothing
#    next_perm!(s) || return (s, nothing)
#    return (copy(s), s)
#end

end

Main.O

In [2]:
using BenchmarkTools

In [3]:
p = O.perm([NaN, NaN, 2, 1])
_, state = iterate(p)

([1.0, 2.0, NaN, NaN], ([1.0, NaN, 2.0, NaN], [1.0, NaN, NaN, 2.0]))

In [4]:
iterate(p, state)

([1.0, NaN, 2.0, NaN], ([1.0, NaN, NaN, 2.0], [2.0, 1.0, NaN, NaN]))

In [5]:
p = O.perm([NaN, NaN, 2, 1])
@btime collect($p)

  853.846 ns (25 allocations: 1.92 KiB)


12-element Vector{Vector{Float64}}:
 [1.0, 2.0, NaN, NaN]
 [1.0, NaN, 2.0, NaN]
 [1.0, NaN, NaN, 2.0]
 [2.0, 1.0, NaN, NaN]
 [2.0, NaN, 1.0, NaN]
 [2.0, NaN, NaN, 1.0]
 [NaN, 1.0, 2.0, NaN]
 [NaN, 1.0, NaN, 2.0]
 [NaN, 2.0, 1.0, NaN]
 [NaN, 2.0, NaN, 1.0]
 [NaN, NaN, 1.0, 2.0]
 [NaN, NaN, 2.0, 1.0]

In [6]:
v = sort([NaN, NaN, 2, 1])
@btime O.collect_perm!($v)

  856.250 ns (15 allocations: 1.81 KiB)


12-element Vector{Vector{Float64}}:
 [1.0, 2.0, NaN, NaN]
 [1.0, NaN, 2.0, NaN]
 [1.0, NaN, NaN, 2.0]
 [2.0, 1.0, NaN, NaN]
 [2.0, NaN, 1.0, NaN]
 [2.0, NaN, NaN, 1.0]
 [NaN, 1.0, 2.0, NaN]
 [NaN, 1.0, NaN, 2.0]
 [NaN, 2.0, 1.0, NaN]
 [NaN, 2.0, NaN, 1.0]
 [NaN, NaN, 1.0, 2.0]
 [NaN, NaN, 2.0, 1.0]

In [7]:
w = sort(v)
@show w
while O.next_perm!(w)
    @show w
end

w = [1.0, 2.0, NaN, NaN]
w = [1.0, NaN, 2.0, NaN]
w = [1.0, NaN, NaN, 2.0]
w = [2.0, 1.0, NaN, NaN]
w = [2.0, NaN, 1.0, NaN]
w = [2.0, NaN, NaN, 1.0]
w = [NaN, 1.0, 2.0, NaN]
w = [NaN, 1.0, NaN, 2.0]
w = [NaN, 2.0, 1.0, NaN]
w = [NaN, 2.0, NaN, 1.0]
w = [NaN, NaN, 1.0, 2.0]
w = [NaN, NaN, 2.0, 1.0]


In [8]:
test_next_perm!(v) = (c = 1; while O.next_perm!(v) c += 1 end; c)
@show s = [1, 1, 2, 2, 2, 3, 4, 5, 6, 6]
@btime test_next_perm!($s)

s = [1, 1, 2, 2, 2, 3, 4, 5, 6, 6] = [1, 1, 2, 2, 2, 3, 4, 5, 6, 6]
  5.509 ms (0 allocations: 0 bytes)


151200

In [9]:
c = @btime collect($(O.perm(s)))
length(c)

  10.358 ms (302402 allocations: 28.84 MiB)


151200

In [10]:
@code_warntype O.next_perm!(s)

MethodInstance for Main.O.next_perm!(::Vector{Int64})
  from next_perm!(v::AbstractVector) in Main.O at In[1]:3
Arguments
  #self#::Core.Const(Main.O.next_perm!)
  v::Vector{Int64}
Locals
  #2::Main.O.var"#2#4"{Vector{Int64}, Int64}
  #1::Main.O.var"#1#3"{Vector{Int64}}
  j::Int64
  i::Int64
  k::Union{Nothing, Int64}
Body::Bool
1 ─       Core.NewvarNode(:(#2))
│         Core.NewvarNode(:(#1))
│         Core.NewvarNode(:(j))
│         Core.NewvarNode(:(i))
│         Core.NewvarNode(:(k))
│   %6  = Main.O.length(v)::Int64
│   %7  = (%6 ≤ 1)::Bool
└──       goto #3 if not %7
2 ─       return false
3 ─ %10 = Main.O.:(var"#1#3")::Core.Const(Main.O.var"#1#3")
│   %11 = Core.typeof(v)::Core.Const(Vector{Int64})
│   %12 = Core.apply_type(%10, %11)::Core.Const(Main.O.var"#1#3"{Vector{Int64}})
│         (#1 = %new(%12, v))
│   %14 = #1::Main.O.var"#1#3"{Vector{Int64}}
│   %15 = Main.O.firstindex(v)::Core.Const(1)
│   %16 = Main.O.lastindex(v)::Int64
│   %17 = (%16 - 1)::Int64
│   %18 = (%15:%17

In [11]:
using OffsetArrays
@show t = OffsetArray([1, 1, 2, 2, 2, 3, 4, 5, 6, 6], -5:4)
@btime collect($(O.perm(t)))

t = OffsetArray([1, 1, 2, 2, 2, 3, 4, 5, 6, 6], -5:4) = [1, 1, 2, 2, 2, 3, 4, 5, 6, 6]
  11.739 ms (302402 allocations: 32.30 MiB)


151200-element Vector{OffsetVector{Int64, Vector{Int64}}}:
 [1, 1, 2, 2, 2, 3, 4, 5, 6, 6]
 [1, 1, 2, 2, 2, 3, 4, 6, 5, 6]
 [1, 1, 2, 2, 2, 3, 4, 6, 6, 5]
 [1, 1, 2, 2, 2, 3, 5, 4, 6, 6]
 [1, 1, 2, 2, 2, 3, 5, 6, 4, 6]
 [1, 1, 2, 2, 2, 3, 5, 6, 6, 4]
 [1, 1, 2, 2, 2, 3, 6, 4, 5, 6]
 [1, 1, 2, 2, 2, 3, 6, 4, 6, 5]
 [1, 1, 2, 2, 2, 3, 6, 5, 4, 6]
 [1, 1, 2, 2, 2, 3, 6, 5, 6, 4]
 [1, 1, 2, 2, 2, 3, 6, 6, 4, 5]
 [1, 1, 2, 2, 2, 3, 6, 6, 5, 4]
 [1, 1, 2, 2, 2, 4, 3, 5, 6, 6]
 ⋮
 [6, 6, 5, 4, 2, 3, 2, 1, 2, 1]
 [6, 6, 5, 4, 2, 3, 2, 2, 1, 1]
 [6, 6, 5, 4, 3, 1, 1, 2, 2, 2]
 [6, 6, 5, 4, 3, 1, 2, 1, 2, 2]
 [6, 6, 5, 4, 3, 1, 2, 2, 1, 2]
 [6, 6, 5, 4, 3, 1, 2, 2, 2, 1]
 [6, 6, 5, 4, 3, 2, 1, 1, 2, 2]
 [6, 6, 5, 4, 3, 2, 1, 2, 1, 2]
 [6, 6, 5, 4, 3, 2, 1, 2, 2, 1]
 [6, 6, 5, 4, 3, 2, 2, 1, 1, 2]
 [6, 6, 5, 4, 3, 2, 2, 1, 2, 1]
 [6, 6, 5, 4, 3, 2, 2, 2, 1, 1]

In [12]:
@btime O.collect_perm!($(sort(t)))

  11.757 ms (151212 allocations: 28.98 MiB)


151200-element Vector{OffsetVector{Int64, Vector{Int64}}}:
 [1, 1, 2, 2, 2, 3, 4, 5, 6, 6]
 [1, 1, 2, 2, 2, 3, 4, 6, 5, 6]
 [1, 1, 2, 2, 2, 3, 4, 6, 6, 5]
 [1, 1, 2, 2, 2, 3, 5, 4, 6, 6]
 [1, 1, 2, 2, 2, 3, 5, 6, 4, 6]
 [1, 1, 2, 2, 2, 3, 5, 6, 6, 4]
 [1, 1, 2, 2, 2, 3, 6, 4, 5, 6]
 [1, 1, 2, 2, 2, 3, 6, 4, 6, 5]
 [1, 1, 2, 2, 2, 3, 6, 5, 4, 6]
 [1, 1, 2, 2, 2, 3, 6, 5, 6, 4]
 [1, 1, 2, 2, 2, 3, 6, 6, 4, 5]
 [1, 1, 2, 2, 2, 3, 6, 6, 5, 4]
 [1, 1, 2, 2, 2, 4, 3, 5, 6, 6]
 ⋮
 [6, 6, 5, 4, 2, 3, 2, 1, 2, 1]
 [6, 6, 5, 4, 2, 3, 2, 2, 1, 1]
 [6, 6, 5, 4, 3, 1, 1, 2, 2, 2]
 [6, 6, 5, 4, 3, 1, 2, 1, 2, 2]
 [6, 6, 5, 4, 3, 1, 2, 2, 1, 2]
 [6, 6, 5, 4, 3, 1, 2, 2, 2, 1]
 [6, 6, 5, 4, 3, 2, 1, 1, 2, 2]
 [6, 6, 5, 4, 3, 2, 1, 2, 1, 2]
 [6, 6, 5, 4, 3, 2, 1, 2, 2, 1]
 [6, 6, 5, 4, 3, 2, 2, 1, 1, 2]
 [6, 6, 5, 4, 3, 2, 2, 1, 2, 1]
 [6, 6, 5, 4, 3, 2, 2, 2, 1, 1]

In [13]:
@code_warntype O.next_perm!(t)

MethodInstance for Main.O.next_perm!(::OffsetVector{Int64, Vector{Int64}})
  from next_perm!(v::AbstractVector) in Main.O at In[1]:3
Arguments
  #self#::Core.Const(Main.O.next_perm!)
  v::OffsetVector{Int64, Vector{Int64}}
Locals
  #2::Main.O.var"#2#4"{OffsetVector{Int64, Vector{Int64}}, Int64}
  #1::Main.O.var"#1#3"{OffsetVector{Int64, Vector{Int64}}}
  j::Int64
  i::Int64
  k::Union{Nothing, Int64}
Body::Bool
1 ─       Core.NewvarNode(:(#2))
│         Core.NewvarNode(:(#1))
│         Core.NewvarNode(:(j))
│         Core.NewvarNode(:(i))
│         Core.NewvarNode(:(k))
│   %6  = Main.O.length(v)::Int64
│   %7  = (%6 ≤ 1)::Bool
└──       goto #3 if not %7
2 ─       return false
3 ─ %10 = Main.O.:(var"#1#3")::Core.Const(Main.O.var"#1#3")
│   %11 = Core.typeof(v)::Core.Const(OffsetVector{Int64, Vector{Int64}})
│   %12 = Core.apply_type(%10, %11)::Core.Const(Main.O.var"#1#3"{OffsetVector{Int64, Vector{Int64}}})
│         (#1 = %new(%12, v))
│   %14 = #1::Main.O.var"#1#3"{OffsetVector{Int6